# Data Preparation: Access Indicators

This notebook prepares Access-related ICT indicators for a selected group of countries.
The objective is to clean, validate, and structure the data for comparative analysis
in the context of school connectivity and the GIGA initiative.

## Scope of the Notebook

This notebook focuses on:
- Loading selected Access indicators from raw ITU datasets
- Filtering data for a predefined set of countries
- Basic data cleaning and validation
- Exporting cleaned datasets for downstream analysis

In [23]:
# =============================
# Import libraries
# =============================
import pandas as pd
from pathlib import Path

In [24]:
# =============================
# Define data directories
# =============================
DATA_DIR = Path("../data/raw")
OUTPUT_DIR = Path("../data/processed")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [25]:
# =============================
# Load Access indicators
# =============================
hh_internet = pd.read_csv(DATA_DIR / "households_with_internet_access_at_home.csv")
fixed_broadband = pd.read_csv(DATA_DIR / "fixed_broadband_subscriptions.csv")
active_mobile_bb = pd.read_csv(DATA_DIR / "active_mobile_broadband_subscriptions.csv")
pop_coverage_mobile = pd.read_csv(DATA_DIR / "population_coverage_mobile_network.csv")

In [26]:
# =============================
# Define selected countries
# =============================
selected_countries = [
    "Ukraine",
    "South Africa",
    "Aruba",
    "Chile",
    "India",
    "Brazil"
]

In [27]:
# =============================
# List of Access indicators
# =============================
indicators_access = {
    "Households with Internet access at home": hh_internet,
    "Fixed broadband subscriptions": fixed_broadband,
    "Active mobile broadband subscriptions": active_mobile_bb,
    "Population coverage by mobile network technology": pop_coverage_mobile
}

In [28]:
# =============================
# Prepare and filter data
# =============================
prepared_access = {}

for name, df in indicators_access.items():
    df_reduced = df[[
        "entityName",
        "entityIso",
        "dataYear",
        "dataValue"
    ]].copy()

    df_reduced.columns = ["country", "iso3", "year", "value"]

    df_reduced["value"] = pd.to_numeric(df_reduced["value"], errors="coerce")

    df_filtered = df_reduced[
        df_reduced["country"].isin(selected_countries)
    ]

    prepared_access[name] = df_filtered

    print(f"{name}: {df_filtered.shape[0]} rows after filtering")
    display(df_filtered.head(3))

Households with Internet access at home: 117 rows after filtering


,country,iso3,year,value
0,Aruba,ABW,2006,36.7
1,Aruba,ABW,2010,63.0
2,Aruba,ABW,2011,66.2


Fixed broadband subscriptions: 264 rows after filtering


,country,iso3,year,value
0,Aruba,ABW,2003,1399.0
1,Aruba,ABW,2004,6963.0
2,Aruba,ABW,2005,12274.0


Active mobile broadband subscriptions: 162 rows after filtering


,country,iso3,year,value
0,Aruba,ABW,2010,0.0
1,Aruba,ABW,2012,21000.0
16,South Africa,ZAF,2010,8700000.0


Population coverage by mobile network technology: 269 rows after filtering


,country,iso3,year,value
0,Aruba,ABW,2005,90.0
1,Aruba,ABW,2006,99.0
2,Aruba,ABW,2007,99.0


In [29]:
# =============================
# Time coverage check
# =============================
coverage_summary = []

for name, df in prepared_access.items():
    for country in selected_countries:
        country_df = df[df["country"] == country]

        if not country_df.empty:
            coverage_summary.append({
                "indicator": name,
                "country": country,
                "start_year": country_df["year"].min(),
                "end_year": country_df["year"].max(),
                "observations": country_df.shape[0]
            })

coverage_df = pd.DataFrame(coverage_summary)
coverage_df

,indicator,country,start_year,end_year,observations
0,Households with Internet access at home,Ukraine,2002,2023,22
1,Households with Internet access at home,South Africa,2002,2023,22
2,Households with Internet access at home,Aruba,2006,2017,6
3,Households with Internet access at home,Chile,2000,2023,24
4,Households with Internet access at home,India,2002,2025,19
5,Households with Internet access at home,Brazil,2001,2024,24
6,Fixed broadband subscriptions,Ukraine,2005,2024,40
7,Fixed broadband subscriptions,South Africa,2002,2024,46
8,Fixed broadband subscriptions,Aruba,2003,2022,30
9,Fixed broadband subscriptions,Chile,2000,2024,50


In [30]:
# =============================
# Missing values assessment
# =============================
missing_summary = []

for name, df in prepared_access.items():
    missing_summary.append({
        "indicator": name,
        "total_rows": df.shape[0],
        "missing_values": df["value"].isna().sum(),
        "missing_percent": round(df["value"].isna().mean() * 100, 2)
    })

missing_df = pd.DataFrame(missing_summary)
missing_df

,indicator,total_rows,missing_values,missing_percent
0,Households with Internet access at home,117,0,0.0
1,Fixed broadband subscriptions,264,0,0.0
2,Active mobile broadband subscriptions,162,0,0.0
3,Population coverage by mobile network technology,269,0,0.0


In [31]:
# =============================
# Final cleaning
# =============================
cleaned_access = {}

for name, df in prepared_access.items():
    df_clean = df.copy()
    df_clean.loc[df_clean["value"] <= 0, "value"] = pd.NA
    cleaned_access[name] = df_clean

In [32]:
# =============================
# Save cleaned datasets
# =============================
for name, df in cleaned_access.items():
    file_name = (
        name.lower()
        .replace(" ", "_")
        .replace("/", "_")
        .replace("-", "_")
    )
    output_path = OUTPUT_DIR / f"{file_name}.csv"
    df.to_csv(output_path, index=False)

    print(f"Saved: {output_path.name}")

Saved: households_with_internet_access_at_home.csv
Saved: fixed_broadband_subscriptions.csv
Saved: active_mobile_broadband_subscriptions.csv
Saved: population_coverage_by_mobile_network_technology.csv


## Notebook Summary

This notebook prepared Access-related ICT indicators for a selected
group of countries. The cleaned datasets provide a reliable foundation
for comparative analysis of school connectivity within the GIGA framework.